# MFCs vs Partners (AOV)

## Executive summary

In this quick project we solve a requisite from Luca Pancari, we want to compute the AOV placed and delivered for the MFCs test. With it, we can compare Control vs Variant

## Plan

Tasks
- [X] Develop query to extract AOV placed and delivered during test
- [X] Compute AOVs
- [X] Compute std

Conclusions
- [X] Anything to add to knowledge
- [X] Any query to add to repo

## Config

In [1]:
# import matplotlib.pyplot as  plt
# import numpy as np
import pandas as pd
# import seaborn as sns
import sys

sys.path.append('c:\\Users\\Jordi Cremades\\Documents\\Repository')

# from utils import dataset_meta_stats
# from utils import dataset_stats
from utils import query_engines

# dms = dataset_meta_stats.DatasetMetaStats() 
# ds = dataset_stats.DatasetStats()

## Task 1: Develop query to extract AOV placed and delivered during test

In [19]:
#query object
q = query_engines.QueryEngines(
    query='aov.sql', 
    params=None,
    load_from_output_file='aov', #with no .csv
    output_file='aov', #with no .csv
    printq=None
)

df = q.query_run_starbust()

In [22]:
# keep only Control and Test groups
cond = df['experiment_group'] != 'out of experiment'
df = df[cond]

df

,city,experiment_group,order_id,order_final_status,customer_id,order_total_purchase_eur,value_original_eur,value_final_eur
0,BEG,Control Group,100367327535,DeliveredStatus,135905034,43.666776,43.666776,43.666776
1,BUC,Control Group,100367006395,DeliveredStatus,52825007,84.106348,84.106348,84.106348
2,BUC,Control Group,100365372087,DeliveredStatus,152276611,7.085552,7.085552,7.085552
3,BCN,Control Group,100366384622,DeliveredStatus,144580531,34.780000,34.780000,34.780000
4,BUC,Control Group,100366525302,DeliveredStatus,33238592,20.504154,20.265360,20.265360
...,...,...,...,...,...,...,...,...
950566,ZAG,Control Group,100275906537,DeliveredStatus,85726392,8.510000,6.330000,6.330000
950567,BUC,Control Group,100275661681,DeliveredStatus,24872294,24.148100,48.296200,48.296200
950568,BUC,Control Group,100275576567,DeliveredStatus,22375099,23.016221,22.454309,23.016221
950569,BCN,Control Group,100276167168,DeliveredStatus,133598933,17.550000,19.500000,17.550000


## Task 2: Compute AOVs

In [23]:
# we compute the values for the AOV for delivered and placed
pivot_df = pd.pivot_table(
    df, 
    index='city', 
    columns='experiment_group', 
    values=['order_id', 'value_original_eur', 'value_final_eur'], 
    aggfunc={'order_id': pd.Series.nunique, 
            'value_original_eur': 'sum', 
            'value_final_eur': 'sum'})

# rename columns
pivot_df.columns = ['_'.join(col).strip() for col in pivot_df.columns.values]

# reset index to not have multi level index
pivot_df = pivot_df.reset_index()

# Rename columns for better readability
pivot_df = pivot_df.rename(columns={
    'order_id_Control': 'Control_order_count',
    'order_id_Test': 'Test_order_count',
    'value_original_eur_Control': 'Control_original_eur_sum',
    'value_original_eur_Test': 'Test_original_eur_sum',
    'value_final_eur_Control': 'Control_final_eur_sum',
    'value_final_eur_Test': 'Test_final_eur_sum'
})

# AOVs computation
pivot_df['aov_delivered_Control Group'] = pivot_df['value_final_eur_Control Group']/pivot_df['order_id_Control Group']
pivot_df['aov_delivered_Test Group'] = pivot_df['value_final_eur_Control Group']/pivot_df['order_id_Control Group']
pivot_df['aov_original_Control Group'] = pivot_df['value_original_eur_Control Group']/pivot_df['order_id_Control Group']
pivot_df['aov_original_Test Group'] = pivot_df['value_original_eur_Control Group']/pivot_df['order_id_Control Group']

pivot_df

,city,order_id_Control Group,order_id_Test Group,value_final_eur_Control Group,value_final_eur_Test Group,value_original_eur_Control Group,value_original_eur_Test Group,aov_delivered_Control Group,aov_delivered_Test Group,aov_original_Control Group,aov_original_Test Group
0,BCN,226785,21676,5.762220e+06,546938.190000,5.981357e+06,579441.260000,25.408294,25.408294,26.374569,26.374569
1,BEG,57096,5517,1.226686e+06,116306.480688,1.272173e+06,122176.312030,21.484620,21.484620,22.281291,22.281291
2,BUC,327742,31996,8.496971e+06,830685.780166,8.723246e+06,855452.636863,25.925791,25.925791,26.616199,26.616199
3,LIS,176533,15126,3.335420e+06,292534.480000,3.506850e+06,314676.070000,18.894034,18.894034,19.865127,19.865127
4,ZAG,32291,3096,5.787622e+05,54543.150000,5.889569e+05,55767.320000,17.923327,17.923327,18.239041,18.239041


In [16]:
pivot_df.to_csv('aovs.csv')

## Task 3: Compute std

In [24]:
df

,city,experiment_group,order_id,order_final_status,customer_id,order_total_purchase_eur,value_original_eur,value_final_eur
0,BEG,Control Group,100367327535,DeliveredStatus,135905034,43.666776,43.666776,43.666776
1,BUC,Control Group,100367006395,DeliveredStatus,52825007,84.106348,84.106348,84.106348
2,BUC,Control Group,100365372087,DeliveredStatus,152276611,7.085552,7.085552,7.085552
3,BCN,Control Group,100366384622,DeliveredStatus,144580531,34.780000,34.780000,34.780000
4,BUC,Control Group,100366525302,DeliveredStatus,33238592,20.504154,20.265360,20.265360
...,...,...,...,...,...,...,...,...
950566,ZAG,Control Group,100275906537,DeliveredStatus,85726392,8.510000,6.330000,6.330000
950567,BUC,Control Group,100275661681,DeliveredStatus,24872294,24.148100,48.296200,48.296200
950568,BUC,Control Group,100275576567,DeliveredStatus,22375099,23.016221,22.454309,23.016221
950569,BCN,Control Group,100276167168,DeliveredStatus,133598933,17.550000,19.500000,17.550000


In [25]:
# Create a pivot table
pivot_df = pd.pivot_table(df, 
                          index='city', 
                          columns='experiment_group', 
                          values=['value_original_eur', 'value_final_eur'], 
                          aggfunc='std')

# Reset index
pivot_df = pivot_df.reset_index()

# Rename columns for better readability
pivot_df.columns = ['city', 'Control_value_original_eur_std', 'Test_value_original_eur_std',
                    'Control_value_final_eur_std', 'Test_value_final_eur_std']

pivot_df

,city,Control_value_original_eur_std,Test_value_original_eur_std,Control_value_final_eur_std,Test_value_final_eur_std
0,BCN,18.996196,18.204822,19.535312,18.992822
1,BEG,15.898849,15.059514,16.421296,15.731460
2,BUC,19.591457,20.417801,20.130270,20.932374
3,LIS,15.033058,15.014470,15.775170,15.931075
4,ZAG,14.664562,15.751183,14.939373,15.917272


In [31]:
pivot_df.to_csv('stds.csv')